In [4]:
# HP-TES sizing
import pandas as pd

## Define Peak Price and Peak load overlap (discharge time) and charge time

Lets say 12 - 6 pm is discharge 8pm to 10 am is charge

## Find HP Sizing Trapezoid

Points start in bottom right, and the perimeter of the sizing trapezoid is drawn clockwise from point 1 to point 2 to point 3, etc.

In [118]:
# times must be 2 digit strings, 24 hour format
charge_st = '20'
charge_et = '10'
charge_duration = 10
discharge_st = '12'
discharge_et = '20'

In [119]:
# formatting data
df = pd.read_csv('Cell 1A_1B Chilled Water Load -data-2022-09-30 00_41_29.csv', index_col= 'Time', parse_dates= True)
df.head()

,1A-Th-Chilled Water
Time,
2021-08-09 13:28:00,2.20 kW
2021-08-09 13:29:00,2.48 kW
2021-08-09 13:33:00,1.75 kW
2021-08-09 13:34:00,1.59 kW
2021-08-09 13:35:00,1.51 kW


In [120]:
def get_kW(str_val):
    str_splt = str_val.split(' ')
    if str_splt[1] == 'kW':
        return float(str_splt[0])
    elif str_splt[1] == 'W':
        return float(str_splt[0])/1000
    elif str_splt[1] == 'mW':
        return float(str_splt[0])/1000000
    elif str_splt[1] == 'µW':
        return float(str_splt[0])/1000000000  
    else:
        print(str_val)
        #return 0 
        raise(Exception('not kW or W or mW or µW'))


In [121]:
df['value'] = df['1A-Th-Chilled Water'].apply(get_kW)
df = df.drop(columns = '1A-Th-Chilled Water')
df.head()

,value
Time,
2021-08-09 13:28:00,2.20
2021-08-09 13:29:00,2.48
2021-08-09 13:33:00,1.75
2021-08-09 13:34:00,1.59
2021-08-09 13:35:00,1.51


In [122]:
peak_ser = df.loc[f'2021-10-09 {discharge_st}:00:00': f'2021-10-10 {discharge_et}:00:00', 'value']
peak_energy = peak_ser.sum()/60
#charge_energy = df.loc[f'2021-10-09 {charge_st}:00:00': f'2021-10-10 {charge_et}:00:00', 'value'].sum()/60

In [129]:
# Point 1 (B) - Size HP for Peak Load

''' 
HP Capacity (kW) is x axis, TES size (kWh) is y axis
Find peak load (kW) and make that the Heat Pump capacity without any TES
This is refered to as Point B
'''
# hp_size_max = df.max().value # not using peak day
hp_size_max = peak_ser.max() # using value in peak period
point_1 = (hp_size_max, 0)
print(point_1)

(6.24, 0)


In [130]:
# Point 2 (between A and B) - Size TES to reduce HP capacity

'''
Potentially unnecessary point

Possible intermediate size: 
    Full cooling load for single Peak price hour within discharge window
    Half the size 
'''
hp_size = 0.5 * hp_size_max

TES_size = peak_ser.loc[peak_ser - hp_size > 0].sum()/60 # load during peak period above what HP can provide

point_2 = (hp_size, TES_size)
print(point_2)



(3.12, 38.563833333333335)


In [140]:
# Point 3 (A) - Minimize HP capacity
'''
Energy needed during peak - HP Capacity * discharge time - TES size = 0 (all energy at peak satisfied) 
hp_size = peak_energy / (charge_time + discharge_time)

This is essentially the bottom line of the trapezoid
'''

size_reduction = -0.005
hp_size = hp_size_max 
for i in range(0,10000):
    # TES_size_prev = TES_size
    # hp_size_prev = hp_size
    hp_size = hp_size + size_reduction * i
    TES_size = peak_ser.loc[peak_ser - hp_size > 0].sum()/60

    if TES_size > (hp_size * charge_duration):
        break

    #size_reduction = (TES_size_prev - TES_size)/(hp_size_prev - hp_size)
point_3 = (hp_size,TES_size)

In [141]:
point_3

(3.4350000000000014, 35.81733333333333)

In [143]:
# Point 4 C - Find TES size to supply all energy during discharge window
''' 
peak energy = HP capacity * charge time
''' 
TES_size = peak_energy
hp_size = TES_size/charge_duration 
point_4 = (hp_size, TES_size)
print(point_4)


(5.664511708333334, 56.64511708333334)


In [ ]:
# Point 5 (D) - Maximum TES Size and maximum HP size

In [ ]:
class HP_sizer():

    def __init__(file):
        self.file = file